In [ ]:
import cv2;
import numpy as np;
from numpy import append
from numpy import empty
import time


#функция для запись в файл вектора
def writeFile(text, name):
    file=open(name, 'w')
    file.write(text)
    file.close()


#функция для преобразования бинарного изображения в виде вектора;
def convertFile(filename):
    image = cv2.imread(filename, 2);
    _, bw_img = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY);
    vector_bw = tuple(np.reshape(bw_img, -1)); #преобразуем в вектор
    #vector_bw = vectorBinImage.tolist()
    first_count, vector_of_counts_black_and_white = compression_to_bw_format(vector_bw)
    return first_count, vector_of_counts_black_and_white, image


def compression_to_bw_format(vector):
    vector_black_and_white = []
    i = white_index = black_index = 0
    first_count =  "BLACK" if not vector[i] else  "WHITE"
    while i < len(vector):
        if not vector[i]:
            try:
                black_index = vector.index(255, i)
            except ValueError:
                black_index = len(vector)
        else:
            try:
                white_index = vector.index(0, i)
            except ValueError:
                white_index = len(vector)
        count = abs(black_index - white_index)
        vector_black_and_white.append(count)
        i += count
    return first_count, vector_black_and_white    

def compression_to_back_bw_format(first_count, bw_vector_black_and_white, img):
    bw_vector = []
    if first_count == "BLACK":
      for i in range(len(bw_vector_black_and_white)):
        bw_vector.extend((255 if i & 1 else 0,) * bw_vector_black_and_white[i])
    else:
        for i in range(len(bw_vector_black_and_white)):
          bw_vector.extend((0 if i & 1 else 255,) * bw_vector_black_and_white[i])
    matrix_from_vector = empty(shape=(0, 0))
    for j in range(len(img)):
        matrix_from_vector = append(matrix_from_vector, bw_vector[len(img[0]) * j:len(img[0]) * (j + 1)])
    matrix_from_vector = matrix_from_vector.reshape(len(img), len(img[0]))
    return matrix_from_vector

def encrypt(vector, key): 
    return tuple(int((item + key) % 65535) for item in vector)


def decrypt(encrvector, key):
    return tuple(int((item - key) % 65535) for item in encrvector)


#частотный анализ
def frequencyAnalysis(vector):
    freqDict={ }
    vectorSet = set(vector)
    for i in vectorSet:
      freqDict[i] = vector.count(i)
    return freqDict





key = 1717

vector = convertFile('.../pyth/sibirskij-kotik3.jpg');
#print(vector); 
writeFile(str(vector), '.../pyth/cat_img.txt')   
vector_encrypt = encrypt(vector[1], key)
#print(vector_encrypt)
writeFile(str(vector_encrypt), '.../pyth/cat_bin_encrypt.txt') 
vector_decrypt = decrypt(vector_encrypt, key)
#print(vector_decrypt)
decrypted_bw_vector = compression_to_back_bw_format(vector[0], vector_decrypt, vector[2])
#print(decrypted_bw_vector)
#writeFile(str(vector_decrypt), '/content/drive/MyDrive/pyth/cat_bin_decrypt.txt')
cv2.imwrite('.../pyth/cat_new_file.jpg', decrypted_bw_vector) 

resultanalis=[]
for i in range(1,11):
        #img = cv2.imread(f'/content/drive/MyDrive/pyth/text{i}.png', 2)
        #_, bw_img = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)
        #bw_vector = tuple(np.reshape(bw_img, -1))
        first_count, vector_black_and_white, img = convertFile(f'.../pyth/text{i}.jpg')
        #print(vector_black_and_white)
        freqdict =frequencyAnalysis(vector_black_and_white)
        res = dict(sorted(freqdict.items(), key=lambda x: x[1], reverse=True)[:10])
        #print(res)
        resultanalis.append(res)

keys= (1002, 1111, 1234)
frequency_dicts=[]
for i in range(1,4):
      first_count, vector_black_and_white, img = convertFile(f'.../pyth/texted{i}.jpg')
      ervector = encrypt(vector_black_and_white, keys[i-1])
      dict_vector_encrypt = frequencyAnalysis(ervector)
      dict_vector_encrypt = dict(sorted(dict_vector_encrypt.items(), key=lambda item: item[1], reverse=True)[:10])
      frequency_dicts.append(dict_vector_encrypt)
      print(dict_vector_encrypt)
      frequency_dict  = frequencyAnalysis(vector_black_and_white)
      frequency_dict = dict(sorted(frequency_dict.items(), key=lambda item: item[1], reverse=True)[:10])
      print(frequency_dict)

      key = tuple(frequency_dict.keys())[0]
      print(key)
      decrypted_vector = None
      start = time.perf_counter()
      for key_encrypted in dict_vector_encrypt.keys():
        #print(key_encrypted)
        decrypted_vector = decrypt(ervector, key_encrypted - key)
        #print(decrypted_vector)
        if decrypted_vector == tuple(vector_black_and_white):
            print(key_encrypted-key)
            stop = time.perf_counter()
            print( "Text ", stop-start)
            break
      vector_bw = compression_to_back_bw_format(first_count, decrypted_vector, img)
      cv2.imwrite(f'.../pyth/text_for_encr_file{i}.jpg', vector_bw)

